In [1]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from urllib.parse import quote
import os


In [4]:
load_dotenv(dotenv_path="password.env")

host=os.getenv("host")
port=3306
user=os.getenv("user")
database=os.getenv("database")
password=os.getenv("password")
password_embedded = quote(password)  

conn = pymysql.connect(
    host=host,
    port=3306,
    user=user,
    database=database,
    password=password)

# Connecting to SQL

connection = create_engine(f"mysql+pymysql://{user}:{password_embedded}@{host}:{port}/{database}")

In [ ]:
inventory = pd.read_csv('https://raw.githubusercontent.com/nicolehychoi/Forge_Project_2/refs/heads/main/inventory.csv')
books = pd.read_csv('https://raw.githubusercontent.com/nicolehychoi/Forge_Project_2/refs/heads/main/books.csv')
authors = pd.read_csv('https://raw.githubusercontent.com/nicolehychoi/Forge_Project_2/refs/heads/main/authors.csv')
patrons = pd.read_csv('https://raw.githubusercontent.com/nicolehychoi/Forge_Project_2/refs/heads/main/patrons.csv')
loans = pd.read_csv('https://raw.githubusercontent.com/nicolehychoi/Forge_Project_2/refs/heads/main/loans.csv')

inventory.to_sql(name="inventory", con=connection, if_exists='replace', index=False)
# books.to_sql(name="books", con=connection, if_exists='replace', index=False)
    # `author_id ` BIGINT seems to be an issue
authors.to_sql(name="authors", con=connection, if_exists='replace', index=False)
patrons.to_sql(name="patrons", con=connection, if_exists='replace', index=False)
loans.to_sql(name="loans", con=connection, if_exists='replace', index=False)

ProgrammingError: (pymysql.err.ProgrammingError) (1166, "Incorrect column name 'author_id '")
[SQL: 
CREATE TABLE books (
	isbn BIGINT, 
	title TEXT, 
	pub_year BIGINT, 
	genre TEXT, 
	overdue_fee_rate FLOAT(53), 
	`author_id ` BIGINT
)

]
(Background on this error at: https://sqlalche.me/e/20/f405)